In [1]:
import requests
import pandas as pd 
from datetime import datetime

import warnings

# Suprimir o SettingWithCopyWarning
warnings.simplefilter(action='ignore')


In [2]:
# Conexão com API

NOTION_API_KEY = 'secret_GHlV2yGtPWvW6TYpDCHAebaP0QmUbLxc7kEN9e6iVwj'

headers = {
    'Authorization': f'Bearer {NOTION_API_KEY}',
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

In [3]:
# Substitua 'ID_DO_BANCO_DE_DADOS' pelo ID do banco de dados que você deseja consultar
database_id_tim = 'e51a95d56d594c16ad7370bb669f21e5'
database_id_kedu = '0e66d739dca040e997a4e86741bb0b64'

In [4]:

# Faça a solicitação GET para consultar o banco de dados
response_tim = requests.post(f'https://api.notion.com/v1/databases/{database_id_tim}/query', headers=headers)
response_kedu = requests.post(f'https://api.notion.com/v1/databases/{database_id_kedu}/query', headers=headers)

In [5]:
# Resposta TIM
if response_tim.status_code == 200:
    response_data = response_tim.json()
    
    if 'results' in response_data:
        # Aplicar a normalização aos dados
        df_tim = pd.json_normalize(response_data, 'results')
        
    else:
        print("A resposta não contém dados de resultado.")
else:
    print(f'Erro ao consultar o banco de dados. Código de status: {response.status_code}')

In [6]:
# Resposta KEDU
if response_kedu.status_code == 200:
    response_data = response_kedu.json()
    
    if 'results' in response_data:
        # Aplicar a normalização aos dados
        df_kedu = pd.json_normalize(response_data, 'results')
        
    else:
        print("A resposta não contém dados de resultado.")
else:
    print(f'Erro ao consultar o banco de dados. Código de status: {response.status_code}')

In [7]:
# Concatenar dfs
df = pd.concat([df_tim,df_kedu])

In [9]:
# df com colunas filtradas
df_filtrado = df[['id','properties.Name.title','properties.Task.multi_select','parent.database_id','properties.Criado por.created_by.name','properties.Data Criação.date.start','properties.Data Criação.date.end','properties.Status.status.name','created_time','properties.Solicitante.multi_select','properties.Requerente.rich_text','url']]

In [10]:
# Função para tratar campos 

def extrair_informacao(lista, chave='text.content'):
    if lista and isinstance(lista, list):
        primeiro_dicionario = lista[0]
        
        if '.' in chave:
            chaves = chave.split('.')
            valor = primeiro_dicionario
            for k in chaves:
                valor = valor.get(k, {})
            return valor
        
        else:
            return primeiro_dicionario.get(chave)
    return None

In [11]:
# Aplicando a função à coluna properties.Name.title
df_filtrado['Tarefa'] = df_filtrado['properties.Name.title'].apply(lambda x: extrair_informacao(x))

# Aplicando a função à coluna 'coluna_multi_select'
df_filtrado['Tipo'] = df_filtrado['properties.Task.multi_select'].apply(lambda x: extrair_informacao(x, 'name'))

In [12]:
# Definindo a Carteira por meio da database id
df_filtrado.loc[df_filtrado['parent.database_id'] == 'e51a95d5-6d59-4c16-ad73-70bb669f21e5','Carteira'] = 'TIM'
df_filtrado.loc[df_filtrado['parent.database_id'] == '0e66d739-dca0-40e9-97a4-e86741bb0b64','Carteira'] = 'KEDU'

# Preenchendo valores padrão para as linhas não cobertas pelas condições
df_filtrado['Carteira'].fillna('Desconhecida', inplace=True)

In [13]:
# Função para converter e formatar a data
def converter_formatar_data(data_str):
    if pd.notna(data_str):  # Verificar se não é NaN
        # Converter a string para datetime usando pd.to_datetime
        data_obj = pd.to_datetime(data_str)

        # Formatando a data conforme desejado (DD/MM/YYYY)
        data_formatada = data_obj.strftime('%d/%m/%Y')

        return data_formatada
    else:
        return None

In [14]:
# Aplicando a função à coluna do DataFrame
df_filtrado['Inicio'] = df_filtrado['properties.Data Criação.date.start'].apply(converter_formatar_data)
df_filtrado['Término'] = df_filtrado['properties.Data Criação.date.end'].apply(converter_formatar_data)
df_filtrado['DATA DE SOLICITAÇÃO'] = df_filtrado['created_time'].apply(converter_formatar_data)
df_filtrado['Previsão'] = df_filtrado['Término']

In [15]:
# Criando colunas adicionais 'Hoje'

df_filtrado['Hoje'] =  datetime.today().strftime('%d/%m/%Y')
df_filtrado['Data e Hora'] =  datetime.today().strftime('%d/%m/%Y %H:%M:%S')

In [16]:
# Função para subtrair datas
def calcular_diferenca(row):
    if pd.notnull(row['Inicio']) and pd.notnull(row['Término']):
        data_ini = pd.to_datetime(row['Inicio'])
        data_fim = pd.to_datetime(row['Término'])
        diff = data_fim - data_ini
        return diff.days
    else:
        return pd.NaT

In [17]:
# Criando colunas adicionais 'DURAÇÃO (em dias)'
df_filtrado['DURAÇÃO (em dias)'] = df_filtrado.apply(calcular_diferenca, axis=1)

In [18]:
# Criando colunas adicionais 'Qtd Concluido'

df_filtrado.loc[df_filtrado['properties.Status.status.name'] == 'Done','Qtd Concluido'] = 1

# Preenchendo valores padrão para as linhas não cobertas pelas condições
df_filtrado['Qtd Concluido'].fillna(0, inplace=True)

In [19]:
# Criando colunas adicionais 'Qtd Concluído no Prazo'

df_filtrado.loc[df_filtrado['DURAÇÃO (em dias)'] >= 0,'Qtd Concluído no Prazo'] = 1
df_filtrado.loc[df_filtrado['properties.Status.status.name'] == 'Done','Qtd Concluído no Prazo'] = 1

# Preenchendo valores padrão para as linhas não cobertas pelas condições
df_filtrado['Qtd Concluído no Prazo'].fillna(0, inplace=True)

In [20]:
# Criando colunas adicionais 'Qtd Concluído Com Atraso'

df_filtrado['Qtd Concluído Com Atraso'] = 0

# Ajuste as linhas onde as condições são atendidas
mask = (df_filtrado['Qtd Concluído no Prazo'] == 0) & (df_filtrado['properties.Status.status.name'] == 'Done')
df_filtrado.loc[mask, 'Qtd Concluído Com Atraso'] = 1

In [21]:
# Criando colunas adicionais 'Qtd Projeto','Qtd status' e 'COMENTÁRIOS'

df_filtrado['Qtd Projeto'] = 1
df_filtrado['Qtd status'] = 1
df_filtrado['COMENTÁRIOS']= pd.NaT

In [22]:
df_filtrado.head()

,id,properties.Name.title,properties.Task.multi_select,parent.database_id,properties.Criado por.created_by.name,properties.Data Criação.date.start,properties.Data Criação.date.end,properties.Status.status.name,created_time,properties.Solicitante.multi_select,...,Previsão,Hoje,Data e Hora,DURAÇÃO (em dias),Qtd Concluido,Qtd Concluído no Prazo,Qtd Concluído Com Atraso,Qtd Projeto,Qtd status,COMENTÁRIOS
0,20c8d0d5-f173-4e84-bee9-cf23760d71d4,"[{'type': 'text', 'text': {'content': 'Acionam...","[{'id': 'pUTA', 'name': 'Demanda', 'color': 'b...",e51a95d5-6d59-4c16-ad73-70bb669f21e5,Jonathan Junio,2023-11-16T17:30:00.000-03:00,None,Homologation,2023-11-17T16:37:00.000Z,"[{'id': 'iKCd', 'name': 'DeepCenter', 'color':...",...,None,21/11/2023,21/11/2023 21:47:26,NaT,0.0,0.0,0,1,1,NaT
1,4cc76d4d-a8c2-47fa-9c5f-62d014ae12f0,"[{'type': 'text', 'text': {'content': 'Reproce...","[{'id': 'kDco', 'name': 'Ajuste', 'color': 're...",e51a95d5-6d59-4c16-ad73-70bb669f21e5,Lucas Gomes Portácio,2023-11-17T11:14:00.000-03:00,None,In progress,2023-11-17T14:14:00.000Z,[{'id': '0361a341-16b3-43ff-a716-32038fe5e393'...,...,None,21/11/2023,21/11/2023 21:47:26,NaT,0.0,0.0,0,1,1,NaT
2,82df31f6-c3f8-4e83-97ce-250e6de34d94,"[{'type': 'text', 'text': {'content': 'Acionam...","[{'id': 'pUTA', 'name': 'Demanda', 'color': 'b...",e51a95d5-6d59-4c16-ad73-70bb669f21e5,Lucas Gomes Portácio,2023-11-16T12:09:00.000-03:00,2023-11-17T12:09:00.000-03:00,Done,2023-11-16T15:08:00.000Z,[{'id': '0361a341-16b3-43ff-a716-32038fe5e393'...,...,17/11/2023,21/11/2023,21/11/2023 21:47:26,1,1.0,1.0,0,1,1,NaT
3,b2b22b8b-ab7f-4080-9af1-43f7e1e6b095,"[{'type': 'text', 'text': {'content': 'Transfe...","[{'id': 'pUTA', 'name': 'Demanda', 'color': 'b...",e51a95d5-6d59-4c16-ad73-70bb669f21e5,Jonathan Junio,2023-11-16T11:00:00.000-03:00,None,Homologation,2023-11-16T15:05:00.000Z,"[{'id': 'pUTA', 'name': 'Cliente', 'color': 'b...",...,None,21/11/2023,21/11/2023 21:47:26,NaT,0.0,0.0,0,1,1,NaT
4,4207534e-bfd3-43ea-a99f-95c1c9cbeaff,"[{'type': 'text', 'text': {'content': 'Discage...",[{'id': 'ec66cd32-4abc-4ffc-9916-d202a74d7afe'...,e51a95d5-6d59-4c16-ad73-70bb669f21e5,Igor Matheus,NaN,NaN,Not started,2023-11-14T17:02:00.000Z,[],...,None,21/11/2023,21/11/2023 21:47:26,NaT,0.0,0.0,0,1,1,NaT


In [23]:
df_filtrado.to_excel('C:/Users/int.lucas/Downloads/api_notion.xlsx',index=False)